In [ ]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
os.listdir('data/training/training')

In [ ]:
train_data_set_path = 'data/training/training'
test_data_set_path =  'data/validation/validation'

In [ ]:
training_transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

In [ ]:
training_dataset = torchvision.datasets.ImageFolder(root = train_data_set_path, transform = training_transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = training_dataset, batch_size = 32, shuffle = False)

In [ ]:
#next(iter(train_loader))

In [ ]:
def get_mean_and_std(loader):
    mean = 0.
    std = 0.
    Total_images_count = 0
    for images, _ in loader:
        image_count_in_a_batch = images.size(0)
        images = images.view(image_count_in_a_batch, images.size(1),-1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        Total_images_count += image_count_in_a_batch
    mean /= Total_images_count
    std  /= Total_images_count

    return mean,std

In [ ]:
get_mean_and_std(train_loader)

In [ ]:
mean = [0.4363, 0.4328, 0.3291]
std = [0.2129, 0.2075, 0.2038]
train_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

test_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root = train_data_set_path, transform =train_transforms )
test_dataset = torchvision.datasets.ImageFolder(root = test_data_set_path, transform = test_transforms)

In [ ]:
def show_transformed_images(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size = 6,shuffle = True)
    batch = next(iter(loader))
    images, labels = batch

    grid = torchvision.utils.make_grid(images, nrow=3)
    plt.figure(figsize=(11,11))
    plt.imshow(np.transpose(grid, (1,2,0)))
    print('labes:',labels)

In [ ]:
show_transformed_images(train_dataset)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [ ]:
def set_device():
    if torch.cuda.is_available():
        #print('cuda gpu is available')
        dev = 'cuda:0'
    else:
        dev = 'cpu'
    return torch.device(dev)     

In [ ]:
set_device()

In [ ]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epoch):
    device = set_device()
    best_acc = 0
    for epoch in range(n_epoch):
        print('Epoch number %d' % (epoch +1))
        model.train()
        running_loss = 0
        running_correct = 0
        total = 0

        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)

            optimizer.zero_grad()
            
            outputs=model(images)
            _, predicted = torch.max(outputs.data, 1)

            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()

        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total

        print(" _Training dataset got %d out of %d images correct (%.3f%%).epoch loss:%.3f " % (running_correct, total, epoch_acc, epoch_loss))

        Test_dataset_acc = evaluate_model_on_test_dataset(model, test_loader)
        if(Test_dataset_acc > best_acc):
            best_acc = Test_dataset_acc
            save_checkpoint(model, epoch, optimizer, best_acc)
        
    print('finished')
    return model

In [ ]:
def evaluate_model_on_test_dataset(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0 
    device  = set_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)

            outputs=model(images)
            _, predicted = torch.max(outputs.data, 1)

            predicted_correctly_on_epoch += (predicted ==labels).sum().item()

    epoch_acc = 100.00 * predicted_correctly_on_epoch / total
    print(" _Testing dataset. got %d out of %d images correctly(%.3f%%)" % (predicted_correctly_on_epoch, total, epoch_acc))

    return epoch_acc

In [ ]:
def save_checkpoint(model, epoch, optimizer, best_acc):
    state = {
        'epoch': epoch + 1,
        'model': model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),

    }
    torch.save(state, 'model_best_checkpoint.pth.tar')

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet18_model = models.resnet18(pretrained = False)
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 10
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
device = set_device()
resnet18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [ ]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 20)

In [ ]:
checkpoint =  torch.load('model_best_checkpoint.pth.tar')

In [ ]:
print(checkpoint['epoch'])
print(checkpoint['best_acc'])

In [32]:

resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 10
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
resnet18_model.load_state_dict(checkpoint['model']) 

torch.save(resnet18_model, 'best_model.pth')